## Setup 

[paper](http://idl.cs.washington.edu/papers/reverse-engineering-vis/) | [pretrained models](https://osf.io/wubdr/) | [darknet](https://github.com/visual-ds/darknet) 

The next cell will install the dependencies and, in general, prepare the environment; it will need (for the data and the pretrained models) approximately 2.5 GB of space. 

In [ ]:
# install dependencies 
!conda env create -f env.yml 

# install darknet 
!git clone https://github.com/visual-ds/darknet 
!cd darknet && make 

import os 

CWD = os.path.abspath(os.getcwd()) 

with open("config.json", "w") as file: 
    config = """{{ 
    "darknet_lib_path": "{path}/./darknet" 
}}\n""".format(path = CWD + "/darknet") 
    file.write(config) 

#pretrained models 
print("Downloading models from https://osf.io/ckj5z/download")  
!wget https://osf.io/ckj5z/download -O models.zip 
!unzip models.zip 

# data 
print("Downloading data from https://osf.io/uwtsv/") 
!wget https://osf.io/uwtsv/download -O data.zip 
!unzip data.zip 


## Pipelines 

Currently, we have two methods for text localization (`craft` and `default`, that is described in the paper) and two methods for text recognition (`attn` and `tesseract`, which is the default). The general guidelines, then, of how to use it are described in the next cells; ideally, they should compute the specficiations for the chart. 

In [ ]:
import sys 
sys.path.append("..") 

In [ ]:
from IPython.display import JSON
from rev.spec.generator import SpecGenerator
import json

from rev.chart import Chart
from rev.text.localizer import TextLocalizer
from rev.text import TextClassifier

attn_parameters = { 
    # "image_folder": "imgboxes", 
    "saved_model": "../models/attn/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth", 
} 

text_classifier = {
    "default": "../models/text_role_classifier/text_type_classifier.pkl" 
} 

# Load a chart 
quartz = "../data/quartz/4JevQPPu.png" # from OSF 
quartzII = "../data/quartz/4J0AakdDl.png" # from OSF 
chartp = "../examples/chart.png" 
area = "../examples/image.png" 

chart = Chart(area) 

# Text localization and recognition:
localizer = TextLocalizer(ocr = "attn", 
                          attn_params = attn_parameters)    

# set textbox information
text_boxes = localizer.localize([chart], debug = True)
chart.text_boxes = text_boxes[0]

# print(chart.text_boxes) 
# Getting the roles for each textbox
text_clf = TextClassifier(model_checkpoint = text_classifier["default"]) 

text_type_preds = text_clf.classify([chart])

# Set the role for each textbox on the chart
for (text_box, role) in zip(chart.text_boxes, text_type_preds[0]):
    text_box.type = role

# Generate specification and chart mark 
spec_gen = SpecGenerator()
spec = spec_gen.generate([chart]) 

json.loads(spec[0]) 

For the Neural Network based techniques, we need to set some hyperparameters: `cuda` is, in general, set to `False`; the others are, in this pipeline, the default. 

In [ ]:
attn_parameters = { 
    # "image_folder": "imgboxes", 
    "saved_model": "../models/attn/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth", 
} 

import torch 
cuda = torch.cuda.is_available() 

craft_params = {
    "text_threshold": .7,
    "link_threshold": .4,
    "low_text": .4,
    "poly": False,
    "canvas_size": 1280,
    "mag_ratio": 1.8,
    "cuda": cuda 
} 

In [ ]:
from rev.chart import Chart 
from rev.text.localizer import TextLocalizer 
from rev.text.classifier import TextClassifier 
from rev.spec.generator import SpecGenerator 

import json 
# import torch 
quartz = "../data/quartz/4JevQPPu.png" # from OSF 
quartzII = "../data/quartz/4J0AakdDl.png" # from OSF 
chartp = "../examples/chart.png" 
area = "../examples/image.png" 
vega = "../data/vega/burtin1/mark=bar|x=Penicilin,O|y=count_*,Q.png"
chart = Chart("../data/quartz/B1LOWeGM.png")   

# cuda = torch.cuda.is_available() 

text_classifier = {
    "default": "../models/text_role_classifier/text_type_classifier.pkl" 
} 

localizer = TextLocalizer("craft", 
                          craft_model = "../models/craft/craft_mlt_25k.pth", 
                          craft_params = craft_params, 
                         ocr = "attn", 
                         attn_params = attn_parameters)  

chart.text_boxes = localizer.localize([chart], 
                                      debug = True)[0] 

# print(chart.text_boxes) 

text_clf = TextClassifier(model_checkpoint = text_classifier["default"]) 
text_type_preds = text_clf.classify([chart])

# Set the role for each textbox on the chart
for (text_box, role) in zip(chart.text_boxes, text_type_preds[0]):
    text_box.type = role

# Generate specification and chart mark  
spec_gen = SpecGenerator()
spec = spec_gen.generate([chart]) 

json.loads(spec[0]) 